In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dill
#from simba.load_data import LoadData
from sklearn.model_selection import train_test_split
from simba.train_utils import TrainUtils
from simba.preprocessor import Preprocessor
import pickle
import sys
from simba.config import Config
from simba.parser import Parser
from datetime import datetime
from simba.loader_saver import LoaderSaver
import pickle
import numpy as np
from simba.mces.mces_computation import MCES
import random 
import os 

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [3]:
from simba.simba.preprocessing_simba import PreprocessingSimba

In [4]:
## PARAMETERS
config = Config()
spectra_path = r"/Users/sebas/projects/data/MassSpecGym.mgf"

In [5]:
config.PREPROCESSING_DIR =  '/Users/sebas/projects/data/temp_simba_preprocessing/'

In [6]:
if config.RANDOM_MCES_SAMPLING:
    subfix=''
else:
    subfix='_exhaustive'

In [7]:
output_pairs_file = config.PREPROCESSING_DIR   +     config.PREPROCESSING_PICKLE_FILE
output_np_indexes_train = config.PREPROCESSING_DIR+f"indexes_tani_mces_train{subfix}.npy"
output_np_indexes_val = config.PREPROCESSING_DIR+f"indexes_tani_mces_val{subfix}.npy"
output_np_indexes_test = config.PREPROCESSING_DIR+f"indexes_tani_mces_test{subfix}.npy"
output_nist_file = config.PREPROCESSING_DIR+f"all_spectrums_nist.pkl"
output_neurips_file = config.PREPROCESSING_DIR+f"all_spectrums_neurips.pkl"
output_spectrums_file = config.PREPROCESSING_DIR+f"all_spectrums_neurips_nist_20240814.pkl"

USE_ONLY_LOW_RANGE=True
high_tanimoto_range = 0 if USE_ONLY_LOW_RANGE else 0.5 # to get more high similarity pairs

In [8]:
print(f"output_file:{output_pairs_file}")
# params
max_number_spectra_neurips = 1000000000
max_number_spectra_nist = 10000000000
#train_molecules = 1 * (10**6)
#val_molecules = 10**5
#test_molecules = 10**5
train_molecules =  1*(10**4)
val_molecules = 1*(10**4)
test_molecules = 1*(10**4)

block_size_nist = 30000
use_tqdm = config.enable_progress_bar
load_nist_spectra = True
load_neurips_spectra = True
load_train_val_test_data = (
    True  # to load previously train, test, val with proper smiles
)
write_data_flag = True

output_file:/Users/sebas/projects/data/temp_simba_preprocessing/edit_distance_neurips_nist_exhaustive.pkl


In [9]:
def write_data(
    file_path,
    all_spectrums_train=None,
    all_spectrums_val=None,
    all_spectrums_test=None,
    molecule_pairs_train=None,
    molecule_pairs_val=None,
    molecule_pairs_test=None,
    uniformed_molecule_pairs_test=None,
):
    dataset = {
        "all_spectrums_train": all_spectrums_train,
        "all_spectrums_val": all_spectrums_val,
        "all_spectrums_test": all_spectrums_test,
        "molecule_pairs_train": molecule_pairs_train,
        "molecule_pairs_val": molecule_pairs_val,
        "molecule_pairs_test": molecule_pairs_test,
        "uniformed_molecule_pairs_test": uniformed_molecule_pairs_test,
    }
    with open(file_path, "wb") as file:
        pickle.dump(dataset, file)


In [10]:
all_spectrums=PreprocessingSimba.load_spectra(spectra_path, config, use_gnps_format=False)

/Users/sebas/projects/data/MassSpecGym.mgf
File name ends with mgf


  0%|▎                                                                                                                                                               | 159139/100000000 [00:41<7:09:53, 3870.80it/s]


We reached the end of the array at index 159139


## Division, training, validation, test

In [11]:
print("Dividing between training, validation and test")
all_spectrums_train, all_spectrums_val, all_spectrums_test = (
        TrainUtils.train_val_test_split_bms(all_spectrums)
    )

Dividing between training, validation and test


In [12]:
all_spectrums_train= all_spectrums_train[0:1000]
all_spectrums_val= all_spectrums_val[0:1000]
all_spectrums_test= all_spectrums_test[0:1000]

In [13]:
print('Validation pairs ...')
molecule_pairs_val = MCES.compute_all_mces_results_unique(
    all_spectrums_val,
    max_combinations=val_molecules,
    use_tqdm=use_tqdm,
    max_mass_diff=config.MAX_MASS_DIFF,
    min_mass_diff=config.MIN_MASS_DIFF,
    high_tanimoto_range=high_tanimoto_range,
    num_workers=config.PREPROCESSING_NUM_WORKERS,
    use_exhaustive=True,
    random_sampling=config.RANDOM_MCES_SAMPLING,
    config=config,
    identifier='_val',
    use_edit_distance=False,
    loaded_molecule_pairs= None,
)

Validation pairs ...
Computing MCES results based on unique smiles
getting metadata
finished getting metadata
creating dummy spectra...
Starting computation of molecule pairs
2025-04-02 11:15:44.083157
Number of workers: 60
No random sampling
Number of split arrays: 1
Size of each array: 39
Size of each sub-array: 1
Processing index_array: 0
Value of COMPUTE_SPECIFIC_PAIRS: False
Compute the mols
Computing fingerprints
Finished fongerprints


Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - Pand

Processing: 12
Processing: 2
Processing: 9
Processing: 38
Processing: 0
Processing: 20
Processing: 32
Processing: 35
Processing: 7
Processing: 11
Processing: 33
Processing: 36
Processing: 3
Processing: 14
Processing: 17
Processing: 37
Processing: 13
Processing: 5
Processing: 22
Processing: 15
Processing: 6
Processing: 10
Processing: 1
Processing: 28
Processing: 27
Processing: 18
Processing: 21
Processing: 30
Processing: 34
Processing: 4
Processing: 16
Processing: 19
Processing: 29
Processing: 23
Processing: 25
Processing: 26
Processing: 24
Processing: 8
Processing: 31


TypeError: MCES() got an unexpected keyword argument 'catch_errors'

In [ ]:
print('Validation pairs ...')
molecule_pairs_val = MCES.compute_all_mces_results_unique(
        all_spectrums_val,
        max_combinations=val_molecules,
        use_tqdm=use_tqdm,
        max_mass_diff=config.MAX_MASS_DIFF,
        min_mass_diff=config.MIN_MASS_DIFF,
        high_tanimoto_range=high_tanimoto_range,
        num_workers=config.PREPROCESSING_NUM_WORKERS,
        use_exhaustive=True,
        random_sampling=config.RANDOM_MCES_SAMPLING,
        config=config,
        identifier='_val',
        use_edit_distance=True,
        loaded_molecule_pairs= None,
    )

In [ ]:
molecule_pairs_val

In [20]:
np_file='/Users/sebas/projects/data/temp_simba_preprocessing/ed_mces_indexes_tani_incremental_val_0.npy'

In [21]:
array=np.load(np_file)

In [22]:
array.shape

(38, 4)

In [25]:
array

array([[ 0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  3.,  9., 11.],
       [ 1.,  5., 12., 20.],
       [ 1.,  6., 12., 20.],
       [ 1.,  7.,  9., 28.],
       [ 1.,  9.,  9., 31.],
       [ 2.,  2.,  0.,  0.],
       [ 2.,  7., 15., 22.],
       [ 3.,  1.,  9., 11.],
       [ 3.,  3.,  0.,  0.],
       [ 3.,  7., 13., 27.],
       [ 3.,  9., 11., 30.],
       [ 4.,  4.,  0.,  0.],
       [ 4.,  7., 14., 20.],
       [ 5.,  1., 12., 20.],
       [ 5.,  5.,  0.,  0.],
       [ 5.,  6.,  0.,  0.],
       [ 5.,  7., 19., 26.],
       [ 5.,  9., 17., 27.],
       [ 6.,  1., 12., 20.],
       [ 6.,  5.,  0.,  0.],
       [ 6.,  6.,  0.,  0.],
       [ 6.,  7., 19., 26.],
       [ 6.,  9., 17., 27.],
       [ 7.,  1.,  9., 28.],
       [ 7.,  2., 15., 22.],
       [ 7.,  3., 13., 27.],
       [ 7.,  4., 14., 20.],
       [ 7.,  5., 19., 26.],
       [ 7.,  6., 19., 26.],
       [ 7.,  7.,  0.,  0.],
       [ 8.,  8.,  0.,  0.],
       [ 9.,  1.,  9., 31.],
       [ 9.,  

In [26]:
pickle_file='/Users/sebas/projects/data/temp_simba_preprocessing/mapping_unique_smiles.pkl'

In [27]:
with open(pickle_file, 'rb') as f:
    data=pickle.load(f)

In [29]:
mol_train =data['molecule_pairs_train']

In [30]:
mol_train.df_smiles

,canon_smiles,indexes,number_indexes,mz,charge,library,inchi,ionmode,bms,superclass,classe,subclass
0,C[C@@H]1C[C@@H]2O[C@@H]2/C=C\C(=O)CC(=O)O1,"[25, 26]",2,197.0803,1,janssen,,none,CC1CC(C)CC2CC2CCC(C)C1,None,None,None
1,CC(=O)O[C@H]1[C@@H]2c3cccc4[nH]cc(c34)C[C@H]2N...,"[37, 38, 39, 40]",4,299.1749,1,janssen,,none,CC(C)CC1C(C)CC(C)C2CC3CCC4CCCC(C43)C12,None,None,None
2,C[C@H]1CCCC(=O)CCC/C=C/c2cc(O)cc(O)c2C(=O)O1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",21,319.1535,1,janssen,,none,CC1CCCCCC2CC(C)CC(C)C2C(C)CC(C)CCC1,None,None,None
3,CC1CCCC(=O)CCCC=Cc2cc(O)cc(O)c2C(=O)O1,"[15, 21]",2,319.1540,1,janssen,,none,CC1CCCCCC2CC(C)CC(C)C2C(C)CC(C)CCC1,None,None,None
4,CC(=O)OC1C(O)C2OC3C=C(C)C(=O)C(O)C3(CO)C1(C)C2...,"[54, 60]",2,355.1370,1,janssen,,none,CCC12C(CC(C)C(C)C1C)CC1C(C)C(CC(C)C)C2(C)C12CC2,None,None,None
5,CC(=O)O[C@@H]1[C@@H](O)[C@H]2O[C@@H]3C=C(C)C(=...,"[49, 50, 51, 52, 53, 55, 56, 57, 58, 59]",10,355.1382,1,janssen,,none,CCC12C(CC(C)C(C)C1C)CC1C(C)C(CC(C)C)C2(C)C12CC2,None,None,None
6,COC1=CC(=O)c2c(O)c3c(c(O)c2C1=O)CC1(C)OC3C[C@H...,"[27, 28, 29, 30, 31, 32, 33, 34, 35, 36]",10,377.0861,1,janssen,,none,CCC1CC(C)C2C(C)C3C4CC(C(C)C)CC(C)(C4)CC3C(C)C2C1C,None,None,None
7,C=CC(C)(C)C1(CC2NC(=O)/C(=C\c3cnc[nH]3)NC2=O)C...,"[23, 24]",2,422.1817,1,janssen,,none,CCC(C)(C)C1(CC2CC(C)C(CC3CCCC3)CC2C)C(C)C(C)C2...,None,None,None
8,C=CC(C)(C)[C@@]12C[C@@H]3C(=O)N/C(=C\c4cn(C(C)...,"[69, 70, 71, 72, 73, 74]",6,488.2651,1,janssen,,none,CCC(C)(C)C12CC3C(C)CC(CC4CCC(C(C)(C)CC(C)C)C4)...,None,None,None
9,CC1=C[C@H]2O[C@@H]3C[C@H]4OC(=O)/C=C\C=C\[C@@H...,"[61, 62, 63, 64, 65, 66, 67, 68]",8,515.2634,1,janssen,,none,CC1CCC23CCC(C)CC(C)CCCC(C(C)C)CCCCC(C)CC4CC(CC...,None,None,None
